# Video Game Sales Analysis
## Identifying Patterns That Determine Game Success

════════════════════════════════════════════════════════════════════════════════

**Project Goal:** Analyze video game sales data to identify patterns and characteristics
that determine commercial success in the gaming industry.

**Analysis Approach:** Data exploration → feature analysis → data preparation →
model evaluation → recommendations

════════════════════════════════════════════════════════════════════════════════

## Environment Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st
from dsr_data_tools import analyze_dataset
from dsr_feature_eng_ml import DataSplits, ModelEvaluation

## 1 Load and Analyze Dataset

In [2]:
games = pd.read_csv('./datasets/games.csv')
games_analysis, recommendations = analyze_dataset(games, generate_recs=True)

Rows: 16715
Duplicate rows: 0

Column           Non-null   Data type   
Name                16713   object      
Platform            16715   object      
Year_of_Release     16446   float64     
Genre               16713   object      
NA_sales            16715   float64     
EU_sales            16715   float64     
JP_sales            16715   float64     
Other_sales         16715   float64     
Critic_Score         8137   float64     
User_Score          10014   object      
Rating               9949   object      

Column:             Name
Data type:          object
Non-null:           16713
N/A count:          2
Unique values:      11559

Numeric values:     1
Non-numeric values: 16712

  Recommendations:
  Recommendation: NON_INFORMATIVE
    Reason: High cardinality object type
    Action: Drop column 'Name'


Column:             Platform
Data type:          object
Non-null:           16715
N/A count:          0
Unique values:      31

Numeric values:     133
Non-numeric values: 1

### 1.1 Data Analysis Summary

## 2&emsp;Data Preparation

### 2.1&emsp;Standardizing Column Names  
>Convert all column names to lowercase.

In [10]:
games.rename(columns=str.lower, inplace=True)

In [11]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16715 non-null  object 
 2   year_of_release  16446 non-null  float64
 3   genre            16713 non-null  object 
 4   na_sales         16715 non-null  float64
 5   eu_sales         16715 non-null  float64
 6   jp_sales         16715 non-null  float64
 7   other_sales      16715 non-null  float64
 8   critic_score     8137 non-null   float64
 9   user_score       10014 non-null  object 
 10  rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


### 2.2&emsp;Data Type Conversion

>Replace any NA cells in the Rating column with 'NR' (Not Rated).

In [ ]:
games.rating = games.rating.fillna('NR')
games

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NR
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NR
...,...,...,...,...,...,...,...,...,...,...,...
16710,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,0.00,0.00,0.01,0.00,NaN,NaN,NR
16711,LMA Manager 2007,X360,2006.0,Sports,0.00,0.01,0.00,0.00,NaN,NaN,NR
16712,Haitaka no Psychedelica,PSV,2016.0,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NR
16713,Spirits & Spells,GBA,2003.0,Platform,0.01,0.00,0.00,0.00,NaN,NaN,NR


>The platform, genre, and rating columns should be categories.  
>These columns contain discrete values.

In [13]:
games = games.astype({'platform': 'category', 'genre': 'category', 'rating': 'category'})
games.info()
games.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             16713 non-null  object  
 1   platform         16715 non-null  category
 2   year_of_release  16446 non-null  float64 
 3   genre            16713 non-null  category
 4   na_sales         16715 non-null  float64 
 5   eu_sales         16715 non-null  float64 
 6   jp_sales         16715 non-null  float64 
 7   other_sales      16715 non-null  float64 
 8   critic_score     8137 non-null   float64 
 9   user_score       10014 non-null  object  
 10  rating           16715 non-null  category
dtypes: category(3), float64(6), object(2)
memory usage: 1.1+ MB


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NR
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NR


In [14]:
games.platform.cat.categories

Index(['2600', '3DO', '3DS', 'DC', 'DS', 'GB', 'GBA', 'GC', 'GEN', 'GG', 'N64',
       'NES', 'NG', 'PC', 'PCFX', 'PS', 'PS2', 'PS3', 'PS4', 'PSP', 'PSV',
       'SAT', 'SCD', 'SNES', 'TG16', 'WS', 'Wii', 'WiiU', 'X360', 'XB',
       'XOne'],
      dtype='object')

In [15]:
games.genre.cat.categories

Index(['Action', 'Adventure', 'Fighting', 'Misc', 'Platform', 'Puzzle',
       'Racing', 'Role-Playing', 'Shooter', 'Simulation', 'Sports',
       'Strategy'],
      dtype='object')

In [16]:
games.rating.cat.categories

Index(['AO', 'E', 'E10+', 'EC', 'K-A', 'M', 'NR', 'RP', 'T'], dtype='object')

>Replace 'tbd' scores with NaN.  
>These scores have not been reported.

In [ ]:
games.user_score = games.user_score.replace('tbd', np.nan)

In [18]:
# Verify that the operation was successful.
games[games.user_score == 'tbd']

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating


>Convert the user_score column to float64.

In [ ]:
games = games.astype({'user_score': 'float64'})
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             16713 non-null  object  
 1   platform         16715 non-null  category
 2   year_of_release  16446 non-null  float64 
 3   genre            16713 non-null  category
 4   na_sales         16715 non-null  float64 
 5   eu_sales         16715 non-null  float64 
 6   jp_sales         16715 non-null  float64 
 7   other_sales      16715 non-null  float64 
 8   critic_score     8137 non-null   float64 
 9   user_score       7590 non-null   float64 
 10  rating           16715 non-null  category
dtypes: category(3), float64(7), object(1)
memory usage: 1.1+ MB


### 2.3&emsp;Handling Missing Values

>Determine the number of missing values for each column.

In [20]:
# Examine missing values
row_count = len(games)
missing_name = games.name.isna().sum()
missing_platform = games.platform.isna().sum()
missing_year = games.year_of_release.isna().sum()
missing_genre = games.genre.isna().sum()
missing_na_sales = games.na_sales.isna().sum()
missing_eu_sales = games.eu_sales.isna().sum()
missing_jp_sales = games.jp_sales.isna().sum()
missing_other_sales = games.other_sales.isna().sum()
missing_critic_score = games.critic_score.isna().sum()
missing_user_score = games.user_score.isna().sum()
missing_rating = games.rating.isna().sum()
print(f'Missing name: {missing_name}')
print(f'Missing platform: {missing_platform}')
print(f'Missing year: {missing_year}')
print(f'Missing genre: {missing_genre}')
print(f'Missing NA sales: {missing_na_sales}')
print(f'Missing EU sales: {missing_eu_sales}')
print(f'Missing JP sales: {missing_jp_sales}')
print(f'Missing Other sales: {missing_other_sales}')
print(f'Missing critic score: {missing_critic_score}')
print(f'Missing user score: {missing_user_score}')
print(f'Missing rating: {missing_rating}')

Missing name: 2
Missing platform: 0
Missing year: 269
Missing genre: 2
Missing NA sales: 0
Missing EU sales: 0
Missing JP sales: 0
Missing Other sales: 0
Missing critic score: 8578
Missing user score: 9125
Missing rating: 0


>Determine the percentage of missing values within each column.

In [21]:
def percentage_of_missing_values(n):
    return n / row_count

In [22]:
# Calculate percentage of missing values
print(f'Missing name: {percentage_of_missing_values(missing_name):.2%}')
print(f'Missing platform: {percentage_of_missing_values(missing_platform):.2%}')
print(f'Missing year: {percentage_of_missing_values(missing_year):.2%}')
print(f'Missing genre: {percentage_of_missing_values(missing_genre):.2%}')
print(f'Missing NA sales: {percentage_of_missing_values(missing_na_sales):.2%}')
print(f'Missing EU sales: {percentage_of_missing_values(missing_eu_sales):.2%}')
print(f'Missing JP sales: {percentage_of_missing_values(missing_jp_sales):.2%}')
print(f'Missing Other sales: {percentage_of_missing_values(missing_other_sales):.2%}')
print(f'Missing critic score: {percentage_of_missing_values(missing_critic_score):.2%}')
print(f'Missing user score: {percentage_of_missing_values(missing_user_score):.2%}')
print(f'Missing rating: {percentage_of_missing_values(missing_rating):.2%}')

Missing name: 0.01%
Missing platform: 0.00%
Missing year: 1.61%
Missing genre: 0.01%
Missing NA sales: 0.00%
Missing EU sales: 0.00%
Missing JP sales: 0.00%
Missing Other sales: 0.00%
Missing critic score: 51.32%
Missing user score: 54.59%
Missing rating: 0.00%


### 2.4&emsp;Analyze patterns in missing values

* The number of rows with a missing name is insignificant.
    * These rows can be included in the analysis because the analysis is not based on the game name.
* The number of rows without a release year is small (269; 1.61%).
    * These rows can be excluded.
    * After removing these rows, convert the column to int64.
* The number of rows without a genre is small (2; 0.01%).
    * These rows can be excluded.
* More than half of the rows do not have a critic score and/or a user score.
    * If the analysis involves these scores, exclude these rows; otherwise, they may be included.

>Remove rows without a release year.  
>The resulting row count should be 16715 - 269 = 16446.

In [ ]:
games = games[~games.year_of_release.isna()]
games

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NR
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NR
...,...,...,...,...,...,...,...,...,...,...,...
16710,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,0.00,0.00,0.01,0.00,NaN,NaN,NR
16711,LMA Manager 2007,X360,2006.0,Sports,0.00,0.01,0.00,0.00,NaN,NaN,NR
16712,Haitaka no Psychedelica,PSV,2016.0,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NR
16713,Spirits & Spells,GBA,2003.0,Platform,0.01,0.00,0.00,0.00,NaN,NaN,NR


>Change data type for year_of_release to int64.

In [24]:
games = games.astype({'year_of_release': 'int64'})
games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16446 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             16444 non-null  object  
 1   platform         16446 non-null  category
 2   year_of_release  16446 non-null  int64   
 3   genre            16444 non-null  category
 4   na_sales         16446 non-null  float64 
 5   eu_sales         16446 non-null  float64 
 6   jp_sales         16446 non-null  float64 
 7   other_sales      16446 non-null  float64 
 8   critic_score     7983 non-null   float64 
 9   user_score       7463 non-null   float64 
 10  rating           16446 non-null  category
dtypes: category(3), float64(6), int64(1), object(1)
memory usage: 1.2+ MB


>Remove rows without a genre.  
>Resulting row count should be 16446 - 2 = 16444.

In [ ]:
games = games[~games.genre.isna()]
games

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NR
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NR
...,...,...,...,...,...,...,...,...,...,...,...
16710,Samurai Warriors: Sanada Maru,PS3,2016,Action,0.00,0.00,0.01,0.00,NaN,NaN,NR
16711,LMA Manager 2007,X360,2006,Sports,0.00,0.01,0.00,0.00,NaN,NaN,NR
16712,Haitaka no Psychedelica,PSV,2016,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NR
16713,Spirits & Spells,GBA,2003,Platform,0.01,0.00,0.00,0.00,NaN,NaN,NR


### 2.5&emsp;Calculate Total Sales

>Calculate total sales across all regions and store in a new column.

In [ ]:
games['total_sales'] = games.na_sales + games.eu_sales + games.jp_sales + games.other_sales
games

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76.0,8.0,E,82.54
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NR,40.24
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E,35.52
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80.0,8.0,E,32.77
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NR,31.38
...,...,...,...,...,...,...,...,...,...,...,...,...
16710,Samurai Warriors: Sanada Maru,PS3,2016,Action,0.00,0.00,0.01,0.00,NaN,NaN,NR,0.01
16711,LMA Manager 2007,X360,2006,Sports,0.00,0.01,0.00,0.00,NaN,NaN,NR,0.01
16712,Haitaka no Psychedelica,PSV,2016,Adventure,0.00,0.00,0.01,0.00,NaN,NaN,NR,0.01
16713,Spirits & Spells,GBA,2003,Platform,0.01,0.00,0.00,0.00,NaN,NaN,NR,0.01
